In [1]:
import pandas as pd
import json
import ast
import re

# Обрабатыввем данные и создаем json файлы для модели

In [34]:
df = pd.read_csv('data/final.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4493 entries, 0 to 4492
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Ссылка на статью     4493 non-null   object 
 1   DOI                  4076 non-null   object 
 2   eLIBRARY ID          4493 non-null   int64  
 3   article_type         4493 non-null   object 
 4   year                 4493 non-null   int64  
 5   language             4493 non-null   object 
 6   title_en             4274 non-null   object 
 7   key_words_en         4232 non-null   object 
 8   abstract_en          4172 non-null   object 
 9   authors_metadata_en  4485 non-null   object 
 10  title_ru             1757 non-null   object 
 11  key_words_ru         1757 non-null   object 
 12  abstract_ru          1717 non-null   object 
 13  authors_metadata_ru  1757 non-null   object 
 14  OECD                 4493 non-null   object 
 15  Название журнала     3632 non-null   o

In [35]:
# исправляем 'русский, английский' на 'английски' и оставляем только русский и английский
df.loc[df.language == 'русский, английский', 'language'] = 'английский'
df = df[df.language.isin(['английский', 'русский'])]
df.language.value_counts()

#df.to_csv('data/final_lang_clean', index=False)

language
английский    2768
русский       1702
Name: count, dtype: int64

In [ ]:
# удаляем строки title_en = None, abstract_en = None, a key_words_en = None запоняем '[]'
df = df.dropna(subset=['title_en', 'abstract_en'])
df.loc[df.key_words_en.isna(), 'key_words_en'] = '[]'
print(f'Ключевые слова отсуствуют для {df[df.key_words_en == '[]'].shape[0]} статей\n')
df.info()

Ключевые слова отсуствуют для 787 статей

<class 'pandas.core.frame.DataFrame'>
Index: 4165 entries, 0 to 4492
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Ссылка на статью     4165 non-null   object 
 1   DOI                  3847 non-null   object 
 2   eLIBRARY ID          4165 non-null   int64  
 3   article_type         4165 non-null   object 
 4   year                 4165 non-null   int64  
 5   language             4165 non-null   object 
 6   title_en             4165 non-null   object 
 7   key_words_en         4165 non-null   object 
 8   abstract_en          4165 non-null   object 
 9   authors_metadata_en  4157 non-null   object 
 10  title_ru             1498 non-null   object 
 11  key_words_ru         1498 non-null   object 
 12  abstract_ru          1498 non-null   object 
 13  authors_metadata_ru  1498 non-null   object 
 14  OECD                 4165 non-null   object 
 15  Н

In [50]:
# очистка текста
def clean_text(text):
    if pd.isna(text):
        return ""
    text = re.sub(r'Unknown node type: \w+', '', text)
    text = ' '.join(text.split())
    return text

# keywords в список
def parse_keywords(kw_str):
    if pd.isna(kw_str) or kw_str.strip() == "":
        return []
    try:
        return ast.literal_eval(kw_str)
    except Exception:
        return [kw_str]

In [ ]:
df_train = df[df['first_OECD'] == 'нет'].copy()
df_test = df[df['first_OECD'] != 'нет'].copy()

columns_name = {
    'eLIBRARY ID': 'ID',
    'title_en': 'title',
    'abstract_en': 'abstract',
    'key_words_en': 'keywords'
}

columns = list(columns_name.keys())

df_json = df_train[columns].rename(columns=columns_name)
df_json['abstract'] = df_json['abstract'].apply(clean_text)
df_json['keywords'] = df_json['keywords'].apply(parse_keywords)

zeroshot = df_json.to_dict(orient='records')
print(f'Размер train df: {df_json.shape[0]}')

with open('zeroshot/data/train.json', 'w', encoding='utf-8') as f:
    json.dump(zeroshot, f, ensure_ascii=False, indent=4)

df_json = df_test[columns].rename(columns=columns_name)
df_json['abstract'] = df_json['abstract'].apply(clean_text)
df_json['keywords'] = df_json['keywords'].apply(parse_keywords)  # точно ли надо???????????

zeroshot = df_json.to_dict(orient='records')
print(f'Размер test df: {df_json.shape[0]}')

with open('zeroshot/data/test.json', 'w', encoding='utf-8') as f:
    json.dump(zeroshot, f, ensure_ascii=False, indent=4)

Размер train df: 2859
Размер test df: 1306


In [13]:
from constants import * 

res = {i: dict(enumerate(values)) for i, values in enumerate(FIRST_LEVEL.values())}

with open('zeroshot/data/zeroshot_topics_2.json', 'w', encoding='utf-8') as f:
    json.dump(res, f, ensure_ascii=False, indent=4)

In [11]:
res = {i: dict(enumerate(values)) for i, values in enumerate(FIRST_LEVEL.values())}
res

{0: {0: 'Mathematics',
  1: 'Computer and information sciences',
  2: 'Physical sciences and astronomy',
  3: 'Chemical sciences',
  4: 'Earth and related environmental sciences',
  5: 'Biological sciences',
  6: 'Other natural sciences'},
 1: {0: 'Civil engineering',
  1: 'Electrical engineering, electronic engineering',
  2: 'Mechanical engineering',
  3: 'Chemical engineering',
  4: 'Materials engineering',
  5: 'Medical engineering',
  6: 'Environmental engineering',
  7: 'Environmental biotechnology',
  8: 'Industrial biotechnology',
  9: 'Nano-technology',
  10: 'Other engineering and technologies'},
 2: {0: 'Basic medical research',
  1: 'Clinical medicine',
  2: 'Health sciences',
  3: 'Other medical sciences'},
 3: {0: 'Agriculture, forestry, fisheries',
  1: 'Animal and dairy science',
  2: 'Veterinary science',
  3: 'Other agricultural sciences'},
 4: {0: 'Psychology',
  1: 'Economics and business',
  2: 'Educational sciences',
  3: 'Sociology',
  4: 'Law',
  5: 'Political s

In [ ]:
FIRST_LEVEL = {'Natural Sciences':['Mathematics', 'Computer and information sciences', 'Physical sciences and astronomy', 'Chemical sciences', 'Earth and related environmental sciences', 'Biological sciences', 'Other natural sciences'],
                 'Engineering and Technology':['Civil engineering', 'Electrical engineering, electronic engineering', 'Mechanical engineering', 'Chemical engineering', 'Materials engineering', 'Medical engineering', 'Environmental engineering', 'Environmental biotechnology', 'Industrial biotechnology', 'Nano-technology', 'Other engineering and technologies'],
                 'Medical and Health Sciences': ['Basic medical research', 'Clinical medicine', 'Health sciences', 'Other medical sciences'],
                 'Agricultural Sciences': ['Agriculture, forestry, fisheries', 'Animal and dairy science', 'Veterinary science', 'Other agricultural sciences'],
                 'Social Sciences': ['Psychology', 'Economics and business', 'Educational sciences', 'Sociology', 'Law', 'Political science', 'Social and economic geography', 'Media and communication', 'Other social sciences'],
                 'Humanities': ['History and archaeology', 'Languages and literature', 'Philosophy, ethics and religion', 'Art', 'Other humanities']}